# Skincare Recommendation Engine


This notebook walks you through the development of a content-based recommendation engine that should take a list of skin metrics/concerns (skin type, tone, acne, blemishes, redness, etc) as input and return several products that might suit the user's skin. 

In [1]:
import numpy as np 
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import heapq


In [2]:
# 'essentials' implies essential facial skincare products
df = pd.read_csv('result.csv')
   
makeup = pd.read_csv('result2.csv')

In [3]:
df.head()

,label,url,brand,name,price,skin type,spf,concern,concern 2,concern 3,key ingredient,formulation
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Absolute Perfect Radiance Skin Lightening Day ...,₹ 79,All,NaN,General Care,NaN,NaN,NaN,Cream
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,Biotique,Bio Morning Nectar Flawless Sustainable Skin M...,₹ 165,All,NaN,Uneven Skin Tone,Hydration,Dark Spots,Honey,Lotion
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,Nivea,Unisex Aloe Hydration Skin Cream 100 ml,₹ 92,All,NaN,Dull Skin,Dryness,General Care,Aloe Vera,Cream
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,VI-JOHN,Women Set of 5 Saffron Fairness Cream Advanced,₹ 187,All,15 to 30,Softening and Smoothening,Softening and Smoothening,Sun Protection,Vitamin E,Cream
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Peach Milk Soft Creme 150g,₹ 192,All,NaN,General Care,NaN,NaN,NaN,Cream


In [4]:
makeup['skin tone'].value_counts()


Light to Medium    317
Fair to Light      109
Medium to Dark      49
Dark to Deep        19
Name: skin tone, dtype: int64

## Data Preprocessing

## Imputation of values

In [5]:
df['label'].value_counts()

face-moisturisers    300
cleanser             300
concealer            300
mask-and-peel        300
foundation           300
sunscreen            272
eye-cream            100
Name: label, dtype: int64

In [6]:
df.isna().sum()

label                0
url                  0
brand               46
name                46
price               46
skin type          309
spf               1452
concern            380
concern 2         1136
concern 3         1319
key ingredient    1159
formulation        384
dtype: int64

In [7]:
df['concern 2'].fillna('', inplace = True)
df['concern 3'].fillna('', inplace = True)
df['concern'] = df['concern'] + ',' + df['concern 2'] + ',' + df['concern 3']


In [8]:
df['concern']

0                                          General Care,,
1                   Uneven Skin Tone,Hydration,Dark Spots
2                          Dull Skin,Dryness,General Care
3       Softening and Smoothening,Softening and Smooth...
4                                          General Care,,
                              ...                        
1867                                       Dark Circles,,
1868                Fine Lines and Wrinkles,Dark Circles,
1869                   Dark Circles,Dark Circles,Eye Bags
1870                                                  NaN
1871                        Dark Circles,Eye Bags,Dryness
Name: concern, Length: 1872, dtype: object

In [9]:
df.drop(columns=['concern 2', 'concern 3', 'spf', 'key ingredient', 'formulation'], inplace = True)
df['concern'].value_counts()

Sun Protection,,                                       202
Dark Spots,,                                            81
General Care,,                                          68
Dark Circles,,                                          51
Waterproof,,                                            43
                                                      ... 
Acne or Blemishes,Pigmentation,Anti-Pollution            1
Lightening,Dark Spots,                                   1
Daily Use,Acne or Blemishes,Fine Lines and Wrinkles      1
Anti-Pollution,Acne or Blemishes,Excess Oil              1
Lightening,Dryness,                                      1
Name: concern, Length: 438, dtype: int64

In [10]:
df2 = df[((df['label'] == 'face-moisturisers') | (df['label'] == 'mask-and-peel') | (df['label'] == 'cleanser') | (df['label'] == 'eye-cream'))]
df2
LABELS = list(df2.label.unique())

In [11]:
df2 = df2[df2['skin type'].isna() == False]
df2.index = [i for i in range(0, len(df2))]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      967 non-null    object
 1   url        967 non-null    object
 2   brand      967 non-null    object
 3   name       967 non-null    object
 4   price      967 non-null    object
 5   skin type  967 non-null    object
 6   concern    939 non-null    object
dtypes: object(7)
memory usage: 60.4+ KB


In [12]:
df2[df2['concern'].isna() == True]['label'].value_counts()

face-moisturisers    10
mask-and-peel         9
cleanser              5
eye-cream             4
Name: label, dtype: int64

In [13]:
df2[df2['label'] ==  'eye-cream']['concern'].value_counts()

Fine Lines and Wrinkles,Dark Circles,Eye Bags    16
Dark Circles,Eye Bags,                           12
Dark Circles,,                                   11
Fine Lines and Wrinkles,Dark Circles,             9
Dark Circles,Dark Circles,Eye Bags                8
Dark Circles,Eye Bags,Dryness                     7
Dark Circles,Eye Bags,Dark Circles                6
Dryness,Dark Circles,                             3
Dark Circles,Dryness,                             3
Fine Lines and Wrinkles,Dryness,                  3
Fine Lines and Wrinkles,,                         3
Dryness,,                                         2
Dryness,Eye Bags,Dark Circles                     2
Fine Lines and Wrinkles,Dark Circles,Dryness      2
Lightening,Dark Circles,Eye Bags                  2
Fine Lines and Wrinkles,Dryness,Eye Bags          1
Dark Circles,Dark Circles,                        1
Dark Circles,Dryness,Eye Bags                     1
Lightening,,                                      1
Lightening,D

In [14]:
df2['concern'] = df2['concern'].str.lower()
# df2[df2['label'] == 'face-moisturisers']['concern'].value_counts()
# df2[df2['label'] == 'mask-and-peel']['concern'].value_counts()
df2[df2['label'] == 'sunscreen']['concern'].value_counts()



Series([], Name: concern, dtype: int64)

In [15]:
top_concerns = {'face-moisturisers':'general care', 'mask-and-peel':'daily use', 'cleanser':'general care', 'eye-cream':'fine lines,wrinkles,dark circles,eye bags' }
entries = len(df2)
for i in range(entries):
    label = df2.iloc[i]['label']
    if pd.isnull(df2.iloc[i]['concern']):
        df2.iloc[i]['concern'] = top_concerns[label]
df2.info()

    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      967 non-null    object
 1   url        967 non-null    object
 2   brand      967 non-null    object
 3   name       967 non-null    object
 4   price      967 non-null    object
 5   skin type  967 non-null    object
 6   concern    967 non-null    object
dtypes: object(7)
memory usage: 60.4+ KB


In [16]:
# df2.to_csv('general_skincare.csv')

In [17]:
df2['brand'] = df2['brand'].str.lower()
df2['name'] = df2['name'].str.lower()
df2['skin type'] = df2['skin type'].str.lower()
df2['concern'] = df2['concern'].str.replace(' and ', ',').str.replace(' or ', ',')
df2

,label,url,brand,name,price,skin type,concern
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,absolute perfect radiance skin lightening day ...,₹ 79,all,"general care,,"
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,biotique,bio morning nectar flawless sustainable skin m...,₹ 165,all,"uneven skin tone,hydration,dark spots"
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,nivea,unisex aloe hydration skin cream 100 ml,₹ 92,all,"dull skin,dryness,general care"
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,vi-john,women set of 5 saffron fairness cream advanced,₹ 187,all,"softening,smoothening,softening,smoothening,su..."
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,peach milk soft creme 150g,₹ 192,all,"general care,,"
...,...,...,...,...,...,...,...
962,eye-cream,https://www.myntra.com/eye-cream/skinkraft/ski...,skinkraft,vitamin c+e under eye gel for pigmented under ...,₹ 449,all,"dark circles,,"
963,eye-cream,https://www.myntra.com/eye-cream/casmara/casma...,casmara,eye-cream 15 ml,₹ 2200,all,"fine lines,wrinkles,dark circles,"
964,eye-cream,https://www.myntra.com/eye-cream/myglamm/mygla...,myglamm,white youthfull hydrating eye cream with water...,₹ 716,all,"dark circles,dark circles,eye bags"
965,eye-cream,https://www.myntra.com/eye-cream/makeup-revolu...,makeup revolution london,hydro bank hydrating & cooling eye balm,₹ 1050,all,"fine lines,wrinkles,dark circles,eye bags"


In [18]:
def concern_elements(comma_sep_concerns):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.rstrip()
            if temp in concerns:
                concerns[temp] += 1
            else:
                concerns[temp] = 1


            
# features
list(df2['skin type'].unique())
concerns = {}
for i in range(entries):
    concern_elements(df2.iloc[i]['concern'])
    

In [19]:
print(sorted(concerns.items(), key =
             lambda kv:(kv[1], kv[0])))   

[('anti acne scarring', 1), ('anti-ageing', 2), ('skin tightening', 2), ('blackheads removal', 4), ('irregular textures', 6), ('skin inflammation', 11), ('oil control', 15), ('redness', 15), ('oily skin', 16), ('skin sagging', 20), ('sun protection', 21), ('lightening', 27), ('tan removal', 48), ('anti-pollution', 60), ('eye bags', 60), ('blackheads', 61), ('whiteheads', 61), ('dark spots', 71), ('pigmentation', 71), ('uneven skin tone', 76), ('excess oil', 90), ('dark circles', 103), ('deep nourishment', 106), ('pore care', 110), ('smoothening', 112), ('softening', 112), ('dryness', 119), ('fine lines', 133), ('wrinkles', 133), ('acne', 155), ('blemishes', 155), ('hydration', 156), ('daily use', 158), ('general care', 158), ('dull skin', 166)]


In [20]:
concerns.pop('anti acne scarring')
concerns.pop('anti-ageing')
concerns.pop('skin tightening')
concerns.pop('blackheads removal')
concerns.pop('irregular textures')
concerns.pop('skin inflammation')
concerns.pop('oil control')
concerns.pop('redness')
concerns.pop('oily skin')
concerns.pop('skin sagging')
concerns.pop('sun protection')
concerns.pop('lightening')
concerns.pop('tan removal')
concerns.pop('excess oil')

# concerns.pop('anti acne scarring')
# concerns['scarring'] = 
# concerns.remo('blackheads removal')
# concerns.remove('dull skin')
# # concerns.remove('dullness')
# concerns.add('dull')
# concerns.remove('oily skin')
# concerns.remove('excess oil')
# concerns.remove('oil control')
# concerns.add('oil')

90

In [21]:
features = list(df2['skin type'].unique()) + list(concerns)
(features)

features = ['normal','dry','oily','combination', 'acne','sensitive','fine lines', 'wrinkles', 'redness', 'dull', 'pore', 'pigmentation', 'blackheads','whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots']

In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      967 non-null    object
 1   url        967 non-null    object
 2   brand      967 non-null    object
 3   name       967 non-null    object
 4   price      967 non-null    object
 5   skin type  967 non-null    object
 6   concern    967 non-null    object
dtypes: object(7)
memory usage: 60.4+ KB


In [23]:
len(features)

18

In [24]:
def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(5, len(features)):
        feature = features[j]
        if feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1
            
    

In [25]:
x = one_hot_encodings[456]

In [26]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)

# get_index_from_name('peach milk soft creme 150g')


In [27]:
# finding the closes data points
def recs_nn(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(index2prod(id))

## Cosine Similarity

In [28]:
# utility functions
def name2index(name):
    return df2[df2["name"]==name].index.tolist()[0]

def index2prod(index):
    return df2.iloc[index]

In [29]:

def wrap(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['skin type'] = info_arr[4]
    result['concern'] = str(info_arr[5]).split(',')
    return result


# recommend top 10 similar items from a category
def recs_cs(vector = None, name = None, label = None, count = 5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]
    
    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2
    
    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['brand', 'name', 'price', 'url','skin type','concern']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

In [31]:
# overall recommendation
def recs_essentials(vector = None, name = None):
#     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
#         print(f"{label}:")
        if name: 
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response
            

In [34]:
# features = ['normal','dry','oily','combination', 'acne','sensitive','fine lines', 'wrinkles', 'redness', 'dull', 'pore ', 'pigmentation', 'blackheads','whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots']
x = [0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0]

y = recs_essentials(x, None)
y

{'face-moisturisers': [{'brand': 'bioayurveda',
   'name': 'black pearl radiance anti-pigmentation face cream 60g',
   'price': '₹ 300',
   'url': 'https://www.myntra.com/face-moisturisers/bioayurveda/bioayurveda-black-pearl-radiance-anti-pigmentation-face-cream-60g/15444640/buy',
   'skin type': 'all',
   'concern': ['sun protection', 'pore care', 'redness']},
  {'brand': 'bioayurveda',
   'name': 'pearl radiance anti-pigmentation face cream 120 g',
   'price': '₹ 500',
   'url': 'https://www.myntra.com/face-moisturisers/bioayurveda/bioayurveda-pearl-radiance-anti-pigmentation-face-cream-120-g/15444666/buy',
   'skin type': 'all',
   'concern': ['sun protection', 'pore care', 'redness']},
  {'brand': 'the body shop',
   'name': 'vitamin e intense moisture cream for dry skin 50 ml',
   'price': '₹ 1058',
   'url': 'https://www.myntra.com/face-moisturisers/the-body-shop/the-body-shop-vitamin-e-intense-moisture-cream-for-dry-skin-50-ml/7576908/buy',
   'skin type': 'dry',
   'concern': [

## Makeup Items

In [32]:
# info
makeup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           790 non-null    object 
 1   url             790 non-null    object 
 2   brand           790 non-null    object 
 3   name            790 non-null    object 
 4   price           790 non-null    object 
 5   skin type       443 non-null    object 
 6   skin tone       494 non-null    object 
 7   spf             108 non-null    object 
 8   concern         379 non-null    object 
 9   concern 2       0 non-null      float64
 10  concern 3       0 non-null      float64
 11  key ingredient  118 non-null    object 
 12  formulation     786 non-null    object 
dtypes: float64(2), object(11)
memory usage: 80.4+ KB


In [33]:
makeup.drop(columns = ['concern 2', 'concern 3', 'key ingredient', 'formulation', 'spf', 'concern'], inplace = True)

In [34]:
makeup.isna().sum()

label          0
url            0
brand          0
name           0
price          0
skin type    347
skin tone    296
dtype: int64

In [35]:
makeup.dropna(subset=['skin tone'], inplace=True)


In [36]:
makeup[makeup['label'] == 'foundation']['skin type'].value_counts()

All            73
Normal         31
Oily            7
Combination     3
Dry             1
Name: skin type, dtype: int64

In [37]:
makeup[makeup['label'] == 'foundation']['skin type'].isna().sum()

82

In [38]:
makeup[makeup['label'] == 'primer']['skin type'].isna().sum()

53

In [39]:
makeup[makeup['label'] == 'primer']['skin type'].value_counts()

All            66
Normal         53
Oily           10
Combination     1
Name: skin type, dtype: int64

In [40]:
makeup[makeup['label'] == 'concealer']['skin type'].isna().sum()

70

In [41]:
makeup[makeup['label'] == 'concealer']['skin type'].value_counts()

All            32
Normal         11
Combination     1
Name: skin type, dtype: int64

In [42]:
makeup['skin type'].fillna('All', inplace = True)

In [43]:
makeup['brand'] = makeup['brand'].str.lower()
makeup['name'] = makeup['name'].str.lower()
makeup['skin type'] = makeup['skin type'].str.lower()
makeup['skin tone'] = makeup['skin tone'].str.lower()


In [44]:
def wrap_makeup(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['skin type'] = info_arr[4]
    result['skin tone'] = info_arr[5]
    return result



def makeup_recommendation(skin_tone, skin_type):
    result = []
    dff = pd.DataFrame()
    dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'foundation')].head(2))
    dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'concealer')].head(2))
    dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'primer')].head(2))
    dff= dff.sample(frac = 1)
    data = dff[['brand', 'name', 'price', 'url', 'skin type', 'skin tone']].to_dict('split')['data']
    for element in data:
        result.append(wrap_makeup(element))
    return result



In [45]:
makeup_recommendation('light to medium', 'normal')

C:\Users\vinit\AppData\Local\Temp\ipykernel_4012\2342876624.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'foundation')].head(2))
C:\Users\vinit\AppData\Local\Temp\ipykernel_4012\2342876624.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'concealer')].head(2))
C:\Users\vinit\AppData\Local\Temp\ipykernel_4012\2342876624.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['labe

[{'brand': 'faces canada',
  'name': 'weightless matte finish mini foundation natural 02 18ml',
  'price': '₹ 169',
  'url': 'https://www.myntra.com/foundation-and-primer/faces-canada/faces-canada-weightless-matte-finish-mini-foundation-natural-02-18ml/14254364/buy',
  'skin type': 'normal',
  'skin tone': 'light to medium'},
 {'brand': 'faces canada',
  'name': 'ultime pro makeup fixer',
  'price': '₹ 486',
  'url': 'https://www.myntra.com/foundation-and-primer/faces-canada/faces-canada-ultime-pro-makeup-fixer/2421530/buy',
  'skin type': 'normal',
  'skin tone': 'light to medium'},
 {'brand': 'wet n wild',
  'name': 'sustainable photo focus natural finish setting spray - seal the deal',
  'price': '₹ 454',
  'url': 'https://www.myntra.com/foundation-and-primer/wet-n-wild/wet-n-wild-sustainable-photo-focus-natural-finish-setting-spray---seal-the-deal/12045972/buy',
  'skin type': 'normal',
  'skin tone': 'light to medium'},
 {'brand': 'loreal',
  'name': 'paris true match super blenda

In [46]:
df2
df2.to_csv('general_skin_care_final.csv')

In [47]:
df2.drop(columns = ['cs'], inplace = True)

In [48]:
df2

,label,url,brand,name,price,skin type,concern
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,absolute perfect radiance skin lightening day ...,₹ 79,all,"general care,,"
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,biotique,bio morning nectar flawless sustainable skin m...,₹ 165,all,"uneven skin tone,hydration,dark spots"
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,nivea,unisex aloe hydration skin cream 100 ml,₹ 92,all,"dull skin,dryness,general care"
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,vi-john,women set of 5 saffron fairness cream advanced,₹ 187,all,"softening,smoothening,softening,smoothening,su..."
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,peach milk soft creme 150g,₹ 192,all,"general care,,"
...,...,...,...,...,...,...,...
962,eye-cream,https://www.myntra.com/eye-cream/skinkraft/ski...,skinkraft,vitamin c+e under eye gel for pigmented under ...,₹ 449,all,"dark circles,,"
963,eye-cream,https://www.myntra.com/eye-cream/casmara/casma...,casmara,eye-cream 15 ml,₹ 2200,all,"fine lines,wrinkles,dark circles,"
964,eye-cream,https://www.myntra.com/eye-cream/myglamm/mygla...,myglamm,white youthfull hydrating eye cream with water...,₹ 716,all,"dark circles,dark circles,eye bags"
965,eye-cream,https://www.myntra.com/eye-cream/makeup-revolu...,makeup revolution london,hydro bank hydrating & cooling eye balm,₹ 1050,all,"fine lines,wrinkles,dark circles,eye bags"


In [49]:
len(df2.to_dict('split')['data'])

967

In [50]:
makeup

,label,url,brand,name,price,skin type,skin tone
0,primer,https://www.myntra.com/foundation-and-primer/l...,lakme,absolute blur perfect makeup primer - mini 10g,₹ 254,all,light to medium
2,primer,https://www.myntra.com/foundation-and-primer/w...,wet n wild,sustainable photo focus natural finish setting...,₹ 454,normal,light to medium
4,primer,https://www.myntra.com/foundation-and-primer/w...,wet n wild,sustainable photo focus matte face primer - pa...,₹ 454,normal,light to medium
5,primer,https://www.myntra.com/foundation-and-primer/f...,faces canada,ultime pro perfecting primer 30ml,₹ 719,all,light to medium
6,primer,https://www.myntra.com/foundation-and-primer/w...,wet n wild,sustainable photo focus dewy face primer - til...,₹ 454,all,light to medium
...,...,...,...,...,...,...,...
760,concealer,https://www.myntra.com/concealer/pac/pac-d31-d...,pac,d31 dark take cover concealer crayon 3.94g,₹ 565,all,dark to deep
762,concealer,https://www.myntra.com/concealer/pac/pac-l11-l...,pac,l11 light take cover concealer crayon 3.49g,₹ 565,all,medium to dark
764,concealer,https://www.myntra.com/concealer/pac/pac-spotl...,pac,spotlight liquid concealer 13 warm honey 5.5 g,₹ 765,all,dark to deep
774,concealer,https://www.myntra.com/concealer/benefit-cosme...,benefit cosmetics,boi-ing high coverage cakeless concealer - 06 ...,₹ 2270,all,fair to light


In [51]:
makeup.to_csv('makeup_test.csv')

In [52]:
df2.to_csv('general_test_final.csv')